### Creating tasks

The event loop only keeps weak references to tasks. 

A task that isn’t referenced elsewhere may get garbage collected at any time, even before it’s done. For reliable “fire-and-forget” background tasks, gather them in a collection (A)

In [ ]:
import asyncio

async def some_coro(param):
    await asyncio.sleep(1)
    print(f"Completed coro with param: {param}")

background_tasks = set() # A 1

for i in range(10):
    task = asyncio.create_task(some_coro(param=i))

    # Add task to the set. This creates a strong reference.
    background_tasks.add(task) # A 2

    # To prevent keeping references to finished tasks forever,
    # make each task remove its own reference from the set after
    # completion:
    task.add_done_callback(background_tasks.discard) # A 3

await asyncio.sleep(0)

Completed coro with param: 0
Completed coro with param: 1
Completed coro with param: 2
Completed coro with param: 3
Completed coro with param: 4
Completed coro with param: 5
Completed coro with param: 6
Completed coro with param: 7
Completed coro with param: 8
Completed coro with param: 9


### Task cancellation

- To cancel a task, call its `cancel()` method. (A)
- This schedules a `CancelledError` to be thrown into the wrapped coroutine on its next opportunity to run. (B)

In [ ]:
import asyncio

async def some_coro(param):
    try:
        await asyncio.sleep(1)
    except asyncio.CancelledError: # B
        print(f"Cancelled coro with param: {param}")
        raise
    print(f"Completed coro with param: {param}")

background_tasks = set()

for i in range(10):
    task = asyncio.create_task(some_coro(param=i))
    background_tasks.add(task)
    task.add_done_callback(background_tasks.discard)

await asyncio.sleep(0)
background_tasks.pop().cancel() # A
background_tasks.pop().cancel() 
background_tasks.pop().cancel() 

True

Cancelled coro with param: 6
Cancelled coro with param: 9
Cancelled coro with param: 5


Completed coro with param: 0
Completed coro with param: 1
Completed coro with param: 2
Completed coro with param: 3
Completed coro with param: 4
Completed coro with param: 7
Completed coro with param: 8


### Task groups

Async context manager (A) that can create task and gives reliable way to wait for all tasks to finish.

- Cancellation: Can be done by having a task raising a ignored exception (B).
- Exception handelling: If a task raises an exception, TaskGroup will cancel the remaining scheduled tasks (C).

In [ ]:
import asyncio

async def some_coro(param):
    try:
        await asyncio.sleep(1)
    except asyncio.CancelledError:
        print(f"Cancelled coro with param: {param}")
        raise
    print(f"Completed coro with param: {param}")


async with asyncio.TaskGroup() as tg: # A 1
    for i in range(10):
        tg.create_task(some_coro(param=i))# A 2

# await asyncio.sleep(0)

Completed coro with param: 0
Completed coro with param: 1
Completed coro with param: 2
Completed coro with param: 3
Completed coro with param: 4
Completed coro with param: 5
Completed coro with param: 6
Completed coro with param: 7
Completed coro with param: 8
Completed coro with param: 9


In [ ]:
import asyncio
from asyncio import TaskGroup

class TerminateTaskGroup(Exception):
    pass

async def force_terminate_task_group():
    raise TerminateTaskGroup()

async def job(task_id, sleep_time):
    print(f'Task {task_id}: start')
    await asyncio.sleep(sleep_time)
    print(f'Task {task_id}: done')

async def main():
    try:
        async with TaskGroup() as group:
            group.create_task(job(1, 0.5))
            group.create_task(job(2, 1.5))
            group.create_task(job(3, 2.5)) # C
            group.create_task(job(4, 3.5))

            await asyncio.sleep(1)

            group.create_task(force_terminate_task_group())
    except* TerminateTaskGroup: # exception group instead of normal exception
        print("Task group terminated.")
        pass# B

await main()

Task 1: start
Task 2: start
Task 3: start
Task 4: start
Task 1: done
Task group terminated.


### asyncio.gather

Coroutine are automatically scheduled as tasks (A).
- Cancellation: 
    - Cancel the gather() to cancel all submitted tasks (B).
    - Cancelling a task raises CancelledError in the gather (C).
- Exception handling
    - return_exceptions: exception in result. (D)
    - not return_exceptions: propagate the first exception and continue the rest. (E)

In [ ]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    L = await asyncio.gather(   
        factorial("A", 2), # A
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)

await main()

Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


### sleep

Suspend current task and allow other to run. (A)

In [11]:
import asyncio
import datetime

async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 1.5
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 0.5) >= end_time:
            break
        await asyncio.sleep(0.5) # A

await display_date()

2025-11-12 22:51:31.563430
2025-11-12 22:51:32.070392
2025-11-12 22:51:32.585707
